In [9]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import util.NNmodel
import util.preprocess

#training config
epochs = 1
batch_size = 72
learning_rate=0.001

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#read data
train_data = np.load('./train.npz')
train_features = train_data['x']#features
train_labels = train_data['y']#labels
train_locations = train_data['locations']#locations
train_times = train_data['times']#times
val_data = np.load('./val.npz')
val_features = val_data['x']#features
val_labels = val_data['y']#labels
val_locations = val_data['locations']#locations
val_times = val_data['times']#times

# dataset
train_dataset = TaxiDataset(train_features, train_labels, train_locations, train_times)
val_dataset = TaxiDataset(val_features, val_labels, val_locations, val_times)
# dataloader
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

# initialize model
model = TaxiLSTM().to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
model.train()
for epoch in range(epochs):
    e_loss = []
    #for _ in tqdm(range(total_step), ncols=80):
    for i, (features, labels, locations, times) in enumerate(train_dataloader):
        features = features.to(torch.float32).to(device)
        labels = labels.to(torch.float32).to(device)

        locations = locations.to(torch.float32).to(device)
        times = times.to(torch.float32).to(device)

        # Forward pass
        outputs = model(features, locations, times)
        loss = torch.sqrt(criterion(outputs, labels))
        e_loss.append(loss.item())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print (f'Epoch [{epoch+1}/{epochs}], Loss: {np.mean(e_loss)}')

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
all_loss = []
with torch.no_grad():
    for features, labels, locations, times in val_dataloader:
        features = features.to(torch.float32).to(device)
        labels = labels.to(torch.float32).to(device)
        locations = locations.to(torch.float32).to(device)
        times = times.to(torch.float32).to(device)
        
        outputs = model(features, locations, times)
        loss = torch.sqrt(criterion(outputs, labels)).item()
        all_loss.append(loss)

    print(f'The Root Main square error of validation set is {np.mean(all_loss)}')

#with open('./time.txt',"a") as f:
#    for n in times:
#        f.write(str(n))
#    f.close()

Epoch [1/1], Loss: 6.793342616558075
The Root Main square error of validation set is 5.851031355857849
